In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# **Open Data QnA: Cache Known Good Queries in Vector Store**

---

This notebook shows how to cahce known good queries in a Vector Store that has already been set up using [1_SetUpVectorStore.ipynb](1_SetUpVectorStore.ipynb). The queries are loaded into the vector store from the csv files (/scripts/known_good_sql.csv)

Supported vector stores: 
- pgvector on PostgreSQL 
- BigQuery vector


The notebook covers the following steps: 
> 1. Clean an existing embeddings table for known good queries (if loading_mode = 'refresh')

> 2. Add known good queries from csv file to the embeddings table in the vector store

## 🚧 **0. Pre-requisites**

Make sure that you have completed the intial setup process using [1_SetUpVectorStore.ipynb](1_SetUpVectorStore.ipynb). If the 1_SetUpVectorStore notebook has been run successfully, the following are set up:
* GCP project and all the required IAM permissions

* **Environment to run the solution**

* Data source and Vector store for the solution


## ⚙️ **1. Retrieve Configuration Parameters**
The notebook will load all the configuration parameters from the `config.ini` file in the root directory. 
Most of these parameters were set in the initial notebook `1_SetUpVectorStore.ipynb` and save to the 'config.ini file.
Use the below cells to retrieve these values and specify additional ones required for this notebook. 

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import configparser
config = configparser.ConfigParser()
config.read(module_path+'/config.ini')

PROJECT_ID = config['GCP']['PROJECT_ID']
VECTOR_STORE = config['CONFIG']['VECTOR_STORE']
PG_DATABASE = config['PGCLOUDSQL']['PG_DATABASE']
PG_USER = config['PGCLOUDSQL']['PG_USER']
PG_REGION = config['PGCLOUDSQL']['PG_REGION'] 
PG_INSTANCE = config['PGCLOUDSQL']['PG_INSTANCE'] 
PG_PASSWORD = config['PGCLOUDSQL']['PG_PASSWORD']
BQ_OPENDATAQNA_DATASET_NAME = config['BIGQUERY']['BQ_OPENDATAQNA_DATASET_NAME']
BQ_LOG_TABLE_NAME = config['BIGQUERY']['BQ_LOG_TABLE_NAME'] 
BQ_DATASET_REGION = config['BIGQUERY']['BQ_DATASET_REGION']

## 🔐 **2. Authenticate and Connect to Google Cloud Project**
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

You can do this within Google Colab or using the Application Default Credentials in the Google Cloud CLI.

In [ ]:
"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Google CLI Auth"""
# !gcloud auth application-default login


import google.auth
import os

credentials, project_id = google.auth.default()

os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

# Configure gcloud.
print(f'Project has been set to {PROJECT_ID}')

## 💾 **3. Cache Knwon Good Queries**

### Format of the Known Good SQL File (known_good_sql.csv)

prompt | sql | user_grouping [3 columns]

prompt ==> Natural Language Question corresponding to query

sql ==> SQL for the user question (Note that the sql should enclosed in quotes and only in single line. Please remove the line  break)

user_grouping ==>This name should exactly  match the user_grouping for Postgres Source or Big Query as data_source_list.csv

In [ ]:
# Find the csv file and load as dataframe
import pandas as pd

current_dir = os.getcwd()
root_dir = os.path.expanduser('~')  # Start at the user's home directory

while current_dir != root_dir:
    for dirpath, dirnames, filenames in os.walk(current_dir):
        config_path = os.path.join(dirpath, 'known_good_sql.csv')
        if os.path.exists(config_path):
            file_path = config_path  # Update root_dir to the found directory
            break  # Stop outer loop once found

    current_dir = os.path.dirname(current_dir)

print("Known Good SQL Found at Path :: "+file_path)

# Load the file
df_kgq = pd.read_csv(file_path)
df_kgq = df_kgq.loc[:, ["prompt", "sql", "user_grouping"]]
df_kgq = df_kgq.dropna()
print(df_kgq)

print('Known Good SQLs Loaded into a Dataframe')

### Specify mode for loading the known good sql

The known good sql can loaded in two modes:
* Append mode: Apended to the existing KGQ in the vector store 
* Refresh mode: Delete the existing KGQ and create of fresh copy from KGQ in known_good_sql.csv file

In [5]:
loading_mode = 'refresh' # Options 'append' or 'refresh'
assert loading_mode in {'append', 'refresh'}, "⚠️ Invalid loading_mode. Must be 'append' and 'refresh'"

In [ ]:
# If you have Known Good Queries, load them to known_good_sql.csv file; 
# These will be used as few shot examples for query generation. 

from embeddings.kgq_embeddings import setup_kgq_table, store_kgq_embeddings

if loading_mode == 'refresh':
    # Delete any old tables and create a new table to KGQ embeddings
    if VECTOR_STORE=='bigquery-vector':
        await(setup_kgq_table(project_id=PROJECT_ID,
                            instance_name=None,
                            database_name=None,
                            schema=BQ_OPENDATAQNA_DATASET_NAME,
                            database_user=None,
                            database_password=None,
                            region=BQ_DATASET_REGION,
                            VECTOR_STORE = VECTOR_STORE
                            ))

    elif VECTOR_STORE=='cloudsql-pgvector':
        await(setup_kgq_table(project_id=PROJECT_ID,
                            instance_name=PG_INSTANCE,
                            database_name=PG_DATABASE,
                            schema=None,
                            database_user=PG_USER,
                            database_password=PG_PASSWORD,
                            region=PG_REGION,
                            VECTOR_STORE = VECTOR_STORE
                            ))


print("Adding Known Good Queries to the Vector store.....")
# Add KGQ to the vector store
if VECTOR_STORE=='bigquery-vector':
    await(store_kgq_embeddings(df_kgq,
                                project_id=PROJECT_ID,
                                instance_name=None,
                                database_name=None,
                                schema=BQ_OPENDATAQNA_DATASET_NAME,
                                database_user=None,
                                database_password=None,
                                region=BQ_DATASET_REGION,
                                VECTOR_STORE = VECTOR_STORE
                                ))

elif VECTOR_STORE=='cloudsql-pgvector':
    await(store_kgq_embeddings(df_kgq,
                                project_id=PROJECT_ID,
                                instance_name=PG_INSTANCE,
                                database_name=PG_DATABASE,
                                schema=None,
                                database_user=PG_USER,
                                database_password=PG_PASSWORD,
                                region=PG_REGION,
                                VECTOR_STORE = VECTOR_STORE
                                ))
print('Done!!')